In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


import tensorflow as tf

In [ ]:
df = pd.read_csv("concrete_data.csv")
df.head(2)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89


In [ ]:
df.columns , df.shape

(Index(['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer',
        'Coarse Aggregate', 'Fine Aggregate', 'Age', 'Strength'],
       dtype='object'),
 (1030, 9))

In [ ]:
df.drop(["Age"] , axis = 1 , inplace = True)
df.columns , df.shape

(Index(['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer',
        'Coarse Aggregate', 'Fine Aggregate', 'Strength'],
       dtype='object'),
 (1030, 8))

**1.  Build a Baseline Model**

*1.1.  Building the Model Architecture and Model compilation*

In [ ]:
def reg_model():

    model = tf.keras.models.Sequential(name = "Sequential_")

    model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(df.shape[1]-1,) , name = "First_Hidden_Layer"))
    model.add(tf.keras.layers.Dense(1 , name = "Output_Layer"))


    optimizer , loss = tf.keras.optimizers.Adam() , tf.keras.losses.MeanSquaredError()
    model.compile(optimizer=optimizer, loss=loss)

    return model

*1.2.  Data Splitting*

In [ ]:
input_features , output_values = df.iloc[:,:-1] , df["Strength"]
X_train, X_test, y_train, y_test = train_test_split(input_features, output_values, test_size=0.30)

X_train.shape , X_test.shape , y_train.shape , y_test.shape

((721, 7), (309, 7), (721,), (309,))

*1.3.  Train the model on the training set for 50 epochs.*

In [ ]:
model = reg_model()

model.fit(X_train, y_train, validation_split=0.2, epochs=50, verbose=1)

Epoch 1/50
18/18 [==============================] - 2s 24ms/step - loss: 106596.9062 - val_loss: 74025.9922
Epoch 2/50
18/18 [==============================] - 0s 6ms/step - loss: 48891.1055 - val_loss: 32776.1836
Epoch 3/50
18/18 [==============================] - 0s 5ms/step - loss: 20835.7266 - val_loss: 15042.3105
Epoch 4/50
18/18 [==============================] - 0s 6ms/step - loss: 10410.8330 - val_loss: 9041.4463
Epoch 5/50
18/18 [==============================] - 0s 5ms/step - loss: 7305.9253 - val_loss: 7112.0273
Epoch 6/50
18/18 [==============================] - 0s 7ms/step - loss: 6332.4976 - val_loss: 6413.9956
Epoch 7/50
18/18 [==============================] - 0s 9ms/step - loss: 5882.3721 - val_loss: 5951.1084
Epoch 8/50
18/18 [==============================] - 0s 12ms/step - loss: 5478.3262 - val_loss: 5558.2554
Epoch 9/50
18/18 [==============================] - 0s 5ms/step - loss: 5104.7539 - val_loss: 5197.2832
Epoch 10/50
18/18 [==============================] - 0

*1.4.  Evaluate the model on the test data & Calculating the Mean Squared Error on this test set.*

In [ ]:
model.evaluate(X_test , y_test)

10/10 [==============================] - 0s 3ms/step - loss: 195.7326


195.73257446289062

In [ ]:
#We have already imported the tf.keras.losses.MeanSquaredError() from tensorflow library. No need to use the one of Sklearn.
y_hat = model.predict(X_test)

mse_loss = tf.keras.losses.MeanSquaredError()
mse_error = mse_loss(y_hat , y_test)
print(f"Mean Squared Error for test data is {mse_error}")

10/10 [==============================] - 0s 3ms/step
Mean Squared Error for test data is 333.6095519334504


1.5.  Repeating steps 1 and 3,  50 times.

In [ ]:
list_mse_errors = []
for _ in range(50):

    _ , X_test, _ , y_test = train_test_split(input_features, output_values, test_size=0.30) # You should not use **random_state** parameter so that the results will not be reproducable after each time we split.
    y_hat = model.predict(X_test , verbose = 0)
    mse_error= tf.keras.losses.MeanSquaredError()(y_hat , y_test)
    list_mse_errors.append(mse_error)

*1.6.  Report the means and the standard deviations of the errors.*

In [ ]:
mean , std = np.mean(list_mse_errors) , np.std(list_mse_errors)
print(f"the mean of error after repeating the process 50 times is {mean}, while the standard deviation is {std}")

the mean of error after repeating the process 50 times is 363.92471383693993, while the standard deviation is 19.25420324714637


**B.  Normalizaing the data and Repeating the first part.**

Before doing this, I want to clarify somthing here. The standard deviation (variance) in errors shows that the model is not stable, if it were the variance of error will be small. To mitigate this problem we propose to normalize data. We suppose that the variance would be reduced and the model would be more stable and less sensitive.

In [ ]:
input_features_norm = (input_features - input_features.mean()) / input_features.std()
input_features_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569


In [ ]:
#After normalization the mean of features is ~zero and the std of them is 1.
input_features_norm.mean().mean() , input_features_norm.std().mean()

(-9.657862427807188e-17, 1.0)

In [ ]:
model = reg_model()
X_train, X_test, y_train, y_test = train_test_split(input_features_norm, output_values, test_size=0.30)
model.fit(X_train, y_train, validation_split=0.2, epochs=50, verbose=1) #If you want to show details set verbose to 1, for more details set it to 2.
print("The loss on the test date is")
model.evaluate(X_test , y_test)

Epoch 1/50
18/18 [==============================] - 1s 11ms/step - loss: 1580.5239 - val_loss: 1417.8820
Epoch 2/50
18/18 [==============================] - 0s 4ms/step - loss: 1565.5822 - val_loss: 1403.3363
Epoch 3/50
18/18 [==============================] - 0s 4ms/step - loss: 1550.7284 - val_loss: 1388.9672
Epoch 4/50
18/18 [==============================] - 0s 4ms/step - loss: 1535.8136 - val_loss: 1374.5167
Epoch 5/50
18/18 [==============================] - 0s 3ms/step - loss: 1520.9089 - val_loss: 1359.6582
Epoch 6/50
18/18 [==============================] - 0s 3ms/step - loss: 1505.1879 - val_loss: 1345.0170
Epoch 7/50
18/18 [==============================] - 0s 4ms/step - loss: 1489.5348 - val_loss: 1329.7526
Epoch 8/50
18/18 [==============================] - 0s 5ms/step - loss: 1473.3312 - val_loss: 1314.1007
Epoch 9/50
18/18 [==============================] - 0s 4ms/step - loss: 1456.4126 - val_loss: 1298.4325
Epoch 10/50
18/18 [==============================] - 0s 4ms/ste

539.6744995117188

In [ ]:
list_mse_errors = []
for _ in range(50):

    _ , X_test, _ , y_test = train_test_split(input_features_norm, output_values, test_size=0.30) # You should not use **random_state** parameter so that the results will not be reproducable after each time we split.
    y_hat = model.predict(X_test , verbose = 0)
    mse_error= tf.keras.losses.MeanSquaredError()(y_hat , y_test)
    list_mse_errors.append(mse_error)

In [ ]:
mean , std = np.mean(list_mse_errors) , np.std(list_mse_errors)
print(f"the mean of error after repeating the process 50 times is {mean}, while the standard deviation is {std}")

the mean of error after repeating the process 50 times is 712.3167104452747, while the standard deviation is 31.267749621893632


Opposed to what we thought, the variance of the errors increased from 23 to 36 instead. And even the mean of errors highly increased.

**C.  Increase the number of epochs.**

In [ ]:
model = reg_model()
X_train, X_test, y_train, y_test = train_test_split(input_features_norm, output_values, test_size=0.30)
model.fit(X_train, y_train, validation_split=0.2, epochs=100, verbose=1)
print("The loss on the test date is")
model.evaluate(X_test , y_test)

Epoch 1/100
18/18 [==============================] - 1s 11ms/step - loss: 1707.7429 - val_loss: 1438.1659
Epoch 2/100
18/18 [==============================] - 0s 4ms/step - loss: 1694.1393 - val_loss: 1426.4060
Epoch 3/100
18/18 [==============================] - 0s 5ms/step - loss: 1681.5850 - val_loss: 1415.4912
Epoch 4/100
18/18 [==============================] - 0s 3ms/step - loss: 1669.8657 - val_loss: 1405.1177
Epoch 5/100
18/18 [==============================] - 0s 4ms/step - loss: 1658.6794 - val_loss: 1395.1631
Epoch 6/100
18/18 [==============================] - 0s 3ms/step - loss: 1648.0942 - val_loss: 1385.5425
Epoch 7/100
18/18 [==============================] - 0s 4ms/step - loss: 1637.5251 - val_loss: 1376.0415
Epoch 8/100
18/18 [==============================] - 0s 3ms/step - loss: 1627.3577 - val_loss: 1366.3496
Epoch 9/100
18/18 [==============================] - 0s 3ms/step - loss: 1617.0662 - val_loss: 1356.7321
Epoch 10/100
18/18 [==============================] - 

567.19384765625

In [ ]:
list_mse_errors = []
for _ in range(50):

    _ , X_test, _ , y_test = train_test_split(input_features_norm, output_values, test_size=0.30) # You should not use **random_state** parameter so that the results will not be reproducable after each time we split.
    y_hat = model.predict(X_test , verbose = 0)
    mse_error= tf.keras.losses.MeanSquaredError()(y_hat , y_test)
    list_mse_errors.append(mse_error)

In [ ]:
mean , std = np.mean(list_mse_errors) , np.std(list_mse_errors)
print(f"the mean of error after repeating the process 50 times is {mean}, while the standard deviation is {std}")

the mean of error after repeating the process 50 times is 815.534927039627, while the standard deviation is 37.10114020344214


Both the  mean and variance of the errors decreased.

**D.  Changing the model architecture and repeating the part B.**

In [ ]:
def reg_model():

    model = tf.keras.models.Sequential(name = "Sequential_")

    model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(df.shape[1]-1,) , name = "First_Hidden_Layer"))
    model.add(tf.keras.layers.Dense(10, activation='relu' , name = "Second_Hidden_Layer"))
    model.add(tf.keras.layers.Dense(10, activation='relu' , name = "Third_Hidden_Layer"))

    model.add(tf.keras.layers.Dense(1 , name = "Output_Layer"))


    optimizer , loss = tf.keras.optimizers.Adam() , tf.keras.losses.MeanSquaredError()
    model.compile(optimizer=optimizer, loss=loss)

    return model

In [ ]:
model = reg_model()
X_train, X_test, y_train, y_test = train_test_split(input_features_norm, output_values, test_size=0.30)
model.fit(X_train, y_train, validation_split=0.2, epochs=50, verbose=0) #If you want to show details set verbose to 1, for more details set it to 2.
print("The loss on the test date is")
model.evaluate(X_test , y_test)

The loss on the test date is
10/10 [==============================] - 0s 2ms/step - loss: 176.9533


176.95327758789062

In [ ]:
list_mse_errors = []
for _ in range(50):

    _ , X_test, _ , y_test = train_test_split(input_features_norm, output_values, test_size=0.30) # You should not use **random_state** parameter so that the results will not be reproducable after each time we split.
    y_hat = model.predict(X_test , verbose = 0)
    mse_error= tf.keras.losses.MeanSquaredError()(y_hat , y_test)
    list_mse_errors.append(mse_error)

In [ ]:
mean , std = np.mean(list_mse_errors) , np.std(list_mse_errors)
print(f"the mean of error after repeating the process 50 times is {mean}, while the standard deviation is {std}")

the mean of error after repeating the process 50 times is 417.75035494031874, while the standard deviation is 18.73385451871988
